<a href="https://colab.research.google.com/github/microprediction/winningnotebooks/blob/main/robust_min_var.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install winning
!pip install precise

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.6 MB/s eta 0:00:00
  Created wheel for collinearity: filename=collinearity-0.6.1-py3-none-any.whl size=4442 sha256=887f2682266d2a4666934ade996097aabc9d5984a0b4183f439d0970b7786153
  Stored in directory: /root/.cache/pip/wheels/0e/b1/ea/957b5269b023343361d1f3182a3bd03f8a6fbc86ce713ce7cb
Successfully built collinearity


## Robust Min Var


Let's devise an ad-hoc portfolio construction method.

In [105]:
from winning.std_calibration import std_ability_implied_state_prices, std_state_price_implied_ability
from scipy.stats import norm
from typing import List
import numpy as np
from precise.skaters.portfoliostatic.diagport import diag_long_port
from precise.skaters.covarianceutil.covfunctions import cov_to_corrcoef, multiply_off_diag
from precise.skaters.portfoliostatic.unitport import unit_port
from precise.skaters.portfoliostatic.weakport import weak_long_port
from precise.skaters.portfoliostatic.hrpport import hrp_diag_diag_s5_long_port


def ability_port(cov, n_samples=100000, phi=0.25):
    w_diag = diag_long_port(cov=cov)
    corr = cov_to_corrcoef(cov)
    return ability_tilt(corr=corr, w=w_diag, n_samples=n_samples, phi=phi)


def ability_tilt(corr: np.ndarray, w: np.ndarray, n_samples: int, phi=0.25) -> np.ndarray:
    """
        Tilt portfolio w using corr matrix
    """
    n_dim = np.shape(corr)[0]
    mu = std_state_price_implied_ability(w)
    scaled_corr =  multiply_off_diag(a=corr, phi=phi) # sca
    correlated_samples = np.random.multivariate_normal(mu, scaled_corr, size=n_samples)
    min_indices = np.argmin(correlated_samples, axis=1)
    counts = np.bincount(min_indices, minlength=n_dim)
    return [c/sum(counts) for c in counts]


import numpy as np

# Example correlation matrix (3x3, positive semi-definite, symmetric)
corr_matrix = np.array([[1.0, -0.75, 0.0],
                        [-0.75, 1.0, 0.0],
                        [0.0, 0.0, 1.0]])

# Example state prices or weights (3-dimensional)
w0 = np.array([0.33333, 0.33333, 0.33333])

# Run the ability tilt function
w1 = ability_tilt(corr_matrix, w0, n_samples=5000)

w2 = unit_port(cov=corr_matrix)

print(w0)
print(w1)
print(w2)


[0.33333 0.33333 0.33333]
[0.3532, 0.3444, 0.3024]
[0.44444444 0.44444444 0.11111111]


## Benchmarking against min-var
(Not necessarily the toughest benchmark)

In [106]:
import numpy as np
from scipy.stats import wishart

def generate_true_corr_matrix(n_dim: int) -> np.ndarray:
    """
    Generate a true correlation matrix using a random covariance matrix
    and normalize it to correlation.
    """
    # Generate a random covariance matrix via Wishart distribution
    cov_matrix = wishart.rvs(df=n_dim + 1, scale=np.eye(n_dim))

    # Convert covariance to correlation matrix
    stddevs = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(stddevs, stddevs)

    return corr_matrix

def simulate_data(corr_matrix: np.ndarray, n_samples: int) -> np.ndarray:
    """
    Simulate data from a multivariate normal distribution with the given correlation matrix.
    """
    n_dim = corr_matrix.shape[0]
    mean = np.zeros(n_dim)  # Mean is zero for all dimensions
    data = np.random.multivariate_normal(mean, corr_matrix, size=n_samples)
    return data
from sklearn.covariance import LedoitWolf, MinCovDet

def estimate_correlation(data: np.ndarray) -> dict:
    """
    Estimate correlation matrix using different methods:
    1. Standard sample covariance
    2. Ledoit-Wolf shrinkage
    3. Minimum Covariance Determinant (robust)
    """
    # Sample covariance method
    sample_cov = np.cov(data, rowvar=False)
    sample_corr = np.corrcoef(data, rowvar=False)

    # Ledoit-Wolf shrinkage method
    lw = LedoitWolf()
    lw_cov = lw.fit(data).covariance_
    lw_corr = lw.covariance_ / np.outer(np.sqrt(np.diag(lw_cov)), np.sqrt(np.diag(lw_cov)))

    # Minimum Covariance Determinant (robust method)
    mcd = MinCovDet()
    mcd_cov = mcd.fit(data).covariance_
    mcd_corr = mcd.covariance_ / np.outer(np.sqrt(np.diag(mcd_cov)), np.sqrt(np.diag(mcd_cov)))

    return {
        'sample_corr': sample_corr,
        'ledoitwolf_corr': lw_corr,
        'mcd_corr': mcd_corr
    }


from precise.skaters.portfoliostatic.unitport import unit_port

def compute_portfolios(corr_matrix: np.ndarray, n_samples: int, port, phi=0.5):
    """
    Compute both unit_port and ability_tilt portfolios.
    """
    # Benchmark
    w_port = port(cov=corr_matrix)

    # Ability Tilt
    w_tilt = ability_port(corr_matrix,n_samples=n_samples, phi=phi)
    print(w_tilt)

    return w_port, w_tilt


def compute_portfolio_variance(w: np.ndarray, true_corr_matrix: np.ndarray) -> float:
    """
    Compute the portfolio variance given the weights and the true correlation matrix.
    """
    return np.array(w).T @ true_corr_matrix @ np.array(w)

In [116]:
def compare_methods(port, n_dim: int, n_data_samples, n_samples: int, n_iterations: int = 100, phi=0.25):
    """
    Compare different methods for portfolio allocation over multiple iterations.
    """
    # Initial equal-weight portfolio
    w0 = np.ones(n_dim) / n_dim

    # Track variances
    variances_unit_port = []
    variances_tilt = []

    for _ in range(n_iterations):
        # Step 1: Generate true correlation matrix
        true_corr_matrix = generate_true_corr_matrix(n_dim)

        for _ in range(5):
            # Step 2: Simulate data
            data = simulate_data(true_corr_matrix, n_data_samples)

            # Step 3: Estimate correlation matrices
            estimates = estimate_correlation(data)

            # Step 4: Compute portfolios
            w_unit_port, w_tilt = compute_portfolios(estimates['sample_corr'], n_samples=n_samples, port=port, phi=phi)

            # Step 5: Compute true portfolio variances
            var_unit_port = compute_portfolio_variance(w_unit_port, true_corr_matrix)
            var_tilt = compute_portfolio_variance(w_tilt, true_corr_matrix)

            # Store the variances
            variances_unit_port.append(var_unit_port)
            variances_tilt.append(var_tilt)

            # Compute and compare the means
            mean_var_unit_port = np.mean(variances_unit_port)
            mean_var_tilt = np.mean(variances_tilt)

        print(f"Average Variance (Benchmark Port): {mean_var_unit_port}")
        print(f"Average Variance (Tilt Abil Port): {mean_var_tilt}")



[0.33333 0.33333 0.33333]
[0.3404, 0.3408, 0.3188]
[0.44444444 0.44444444 0.11111111]


In [ ]:
# hrp_diag_diag_s5_long_port
from precise.skaters.portfoliostatic.equalport import equal_long_port
compare_methods(n_dim=500, n_data_samples=300, n_samples=50000, port=hrp_diag_diag_s5_long_port, phi=1.0)

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:748: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


[0.00216, 0.00206, 0.00202, 0.00212, 0.00198, 0.00196, 0.00158, 0.00208, 0.00182, 0.00222, 0.00214, 0.0028, 0.00194, 0.00164, 0.00198, 0.00216, 0.00216, 0.00198, 0.00162, 0.00182, 0.00252, 0.00166, 0.00212, 0.00156, 0.00258, 0.00184, 0.00208, 0.00194, 0.002, 0.00204, 0.00218, 0.00214, 0.00188, 0.00202, 0.00184, 0.00202, 0.0021, 0.00188, 0.00208, 0.0024, 0.0022, 0.0021, 0.00192, 0.00196, 0.0021, 0.002, 0.0022, 0.00176, 0.00218, 0.00202, 0.0019, 0.0018, 0.00208, 0.0019, 0.0023, 0.00196, 0.00238, 0.0022, 0.00206, 0.00202, 0.00218, 0.00212, 0.00222, 0.00206, 0.00222, 0.00184, 0.00206, 0.0021, 0.00196, 0.00236, 0.00188, 0.00198, 0.0023, 0.00182, 0.0017, 0.00228, 0.0019, 0.0018, 0.00184, 0.00194, 0.00212, 0.00186, 0.00188, 0.002, 0.00204, 0.00228, 0.0018, 0.0021, 0.00182, 0.00208, 0.00176, 0.00202, 0.00212, 0.00272, 0.002, 0.00204, 0.00228, 0.00196, 0.00182, 0.00178, 0.00194, 0.0021, 0.00172, 0.00216, 0.00178, 0.00246, 0.00174, 0.00194, 0.00186, 0.0021, 0.00186, 0.00206, 0.002, 0.00184, 0.00

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:748: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


[0.00202, 0.00218, 0.00194, 0.00186, 0.00198, 0.00254, 0.00216, 0.0018, 0.00216, 0.00204, 0.00214, 0.0017, 0.00248, 0.0021, 0.00186, 0.00176, 0.002, 0.00184, 0.0024, 0.00176, 0.00182, 0.00204, 0.002, 0.00212, 0.0021, 0.00244, 0.00202, 0.00204, 0.00194, 0.00194, 0.00206, 0.0017, 0.00218, 0.0021, 0.00186, 0.002, 0.00224, 0.00198, 0.00216, 0.00188, 0.00172, 0.00194, 0.00226, 0.00188, 0.002, 0.00198, 0.00206, 0.0018, 0.00204, 0.00204, 0.00216, 0.00186, 0.00206, 0.00206, 0.00186, 0.00158, 0.00212, 0.00236, 0.002, 0.00202, 0.00162, 0.00186, 0.0019, 0.00182, 0.00196, 0.00194, 0.0019, 0.002, 0.00182, 0.00214, 0.00174, 0.0019, 0.00196, 0.00226, 0.00176, 0.00214, 0.00194, 0.00188, 0.0021, 0.00222, 0.0019, 0.00182, 0.00184, 0.00204, 0.00196, 0.0021, 0.00232, 0.002, 0.0019, 0.00208, 0.0021, 0.00242, 0.00186, 0.00188, 0.0021, 0.00206, 0.0018, 0.002, 0.00182, 0.00162, 0.00196, 0.0021, 0.00192, 0.00232, 0.00226, 0.00212, 0.00226, 0.00202, 0.00196, 0.00176, 0.0023, 0.002, 0.00148, 0.00212, 0.0018, 0.0

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:748: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:748: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


[0.002, 0.00218, 0.00226, 0.00172, 0.00178, 0.00174, 0.00206, 0.00176, 0.00192, 0.00194, 0.00202, 0.00178, 0.0017, 0.00198, 0.0019, 0.00192, 0.00194, 0.00236, 0.0022, 0.00204, 0.00178, 0.00188, 0.002, 0.00196, 0.00228, 0.0022, 0.00204, 0.0019, 0.00186, 0.00218, 0.00178, 0.00184, 0.00238, 0.0021, 0.00186, 0.0019, 0.00206, 0.002, 0.00184, 0.00238, 0.00246, 0.0021, 0.00244, 0.00208, 0.00234, 0.00208, 0.0017, 0.00216, 0.00212, 0.00194, 0.00194, 0.00172, 0.0021, 0.00194, 0.0022, 0.00196, 0.00184, 0.00218, 0.00168, 0.00186, 0.00182, 0.00188, 0.00196, 0.00214, 0.00244, 0.00206, 0.00194, 0.00224, 0.00198, 0.0022, 0.00212, 0.00218, 0.00202, 0.00192, 0.00206, 0.00186, 0.00212, 0.00152, 0.00194, 0.00194, 0.00234, 0.00168, 0.00166, 0.00202, 0.00232, 0.00172, 0.0021, 0.0017, 0.0019, 0.00212, 0.00206, 0.0022, 0.00178, 0.00204, 0.00238, 0.00214, 0.00204, 0.00226, 0.00192, 0.00178, 0.00202, 0.00186, 0.00194, 0.00182, 0.00204, 0.0024, 0.00174, 0.00192, 0.00212, 0.002, 0.00224, 0.00216, 0.00182, 0.00172

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:748: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(


[0.00194, 0.00256, 0.00192, 0.00214, 0.00196, 0.00186, 0.00184, 0.0019, 0.0017, 0.00234, 0.00158, 0.0022, 0.00172, 0.00206, 0.0023, 0.0016, 0.00218, 0.00182, 0.00202, 0.00202, 0.0017, 0.00178, 0.0018, 0.00256, 0.00206, 0.00162, 0.0019, 0.00192, 0.00174, 0.00238, 0.00248, 0.00246, 0.00176, 0.0019, 0.00196, 0.0018, 0.00214, 0.00212, 0.00228, 0.00212, 0.00188, 0.00198, 0.0019, 0.00198, 0.00188, 0.002, 0.00206, 0.00238, 0.00218, 0.00234, 0.00186, 0.00192, 0.0018, 0.002, 0.00172, 0.00204, 0.00214, 0.00218, 0.00208, 0.0023, 0.0019, 0.0017, 0.00162, 0.00224, 0.00218, 0.00224, 0.00212, 0.00174, 0.00168, 0.00196, 0.00196, 0.00198, 0.00186, 0.0023, 0.002, 0.00218, 0.0022, 0.00206, 0.0025, 0.00234, 0.00202, 0.00172, 0.00202, 0.00182, 0.00206, 0.00176, 0.002, 0.00204, 0.00208, 0.00168, 0.0017, 0.00226, 0.00224, 0.00204, 0.00214, 0.00216, 0.0019, 0.00178, 0.00178, 0.002, 0.00204, 0.00208, 0.00208, 0.00188, 0.00224, 0.00214, 0.00192, 0.0022, 0.00222, 0.00222, 0.00202, 0.0019, 0.00182, 0.00194, 0.001

/usr/local/lib/python3.10/dist-packages/sklearn/covariance/_robust_covariance.py:748: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
